# Repositorio Scraping

# TODO

- Build paper catalog

- For each page in paper catalog, get links to papers
    
- get paper_data

In [1]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import sqlite3
import time

# Add to python path to import scripts
sys.path.append('../scripts/')

from src.sqlite import insert_urls, insert_html

## Create Tables

In [2]:
conn = sqlite3.connect("../saopaulo.db")
c = conn.cursor()

c.execute("DROP TABLE IF EXISTS catalog")
c.execute("""CREATE TABLE IF NOT EXISTS catalog (
          id INTEGER PRIMARY KEY,
          html TEXT,
          date_created TEXT,
          current INTEGER,
          parsed INTEGER
         )""")

c.execute("DROP TABLE IF EXISTS urls")
c.execute("""CREATE TABLE IF NOT EXISTS urls (
          id INTEGER PRIMARY KEY,
          items TEXT,
          url_stem TEXT UNIQUE,
          date_created TEXT,
          current INTEGER,
          url_scraped INTEGER
         )""")

conn.commit()
conn.close()

## Get Catalog HTMLs

In [12]:
# Helper functions
async def fetch_one(session, url, conn=None):
    async with session.get(url) as response:
        html_doc = await response.text()
    out_file = url.split("&page=")[1]
    with open(f"../output/{out_file}.html", "w") as f:
        f.write(html_doc)
        # insert_html(html_doc, date, conn)

async def fetch_all(urls, date, db="saopaulo.db"):
    # conn = sqlite3.connect(db)

    async with aiohttp.ClientSession() as session:
        tasks = (fetch_one(session, url) for url in urls)
        await asyncio.gather(*tasks)
        
    # conn.close()

        
# Execute
url_number = 300
date = '20220604'
url_root = 'https://repositorio.usp.br/result.php?filter%5B0%5D=unidadeUSP%3A%22EP%22&fields%5B0%5D=name&fields%5B1%5D=author.person.name&fields%5B2%5D=authorUSP.name&fields%5B3%5D=about&fields%5B4%5D=description&fields%5B5%5D=unidadeUSP&page='
urls = [url_root + str(i) for i in range(url_number)]
db = "../saopaulo.db"

t1 = time.perf_counter()
await fetch_all(urls, date, db=db)
t2 = time.perf_counter()
print(f"Took {t2-t1:.0f} seconds")

Took 37 seconds


In [9]:
urls[0].split("&page=")[1]

'0'

## Parse HTMLs and extract paper_urls

In [7]:
paper_urls = set()
for catalog_html in catalog_htmls:
    soup = BeautifulSoup(catalog_html, 'html5lib')
    articles = soup.find_all('article')
    urls = [article.p.a['href'] for article in articles]
    paper_urls.update(set(urls))

In [8]:
paper_urls

{'item/003005142',
 'item/003012917',
 'item/003017339',
 'item/003018139',
 'item/003021037',
 'item/003021995',
 'item/003027712',
 'item/003027997',
 'item/003030058',
 'item/003030232',
 'item/003032467',
 'item/003032641',
 'item/003032689',
 'item/003033160',
 'item/003033431',
 'item/003035630',
 'item/003036069',
 'item/003036957',
 'item/003036991',
 'item/003037225',
 'item/003037737',
 'item/003037740',
 'item/003038070',
 'item/003038722',
 'item/003041283',
 'item/003041752',
 'item/003043142',
 'item/003043299',
 'item/003043452',
 'item/003043528',
 'item/003043792',
 'item/003044027',
 'item/003047599',
 'item/003049837',
 'item/003051435',
 'item/003054697',
 'item/003055454',
 'item/003056199',
 'item/003056478',
 'item/003057597',
 'item/003057615',
 'item/003058334',
 'item/003058375',
 'item/003058439',
 'item/003058452',
 'item/003060366',
 'item/003060419',
 'item/003060436',
 'item/003060475',
 'item/003060664',
 'item/003061243',
 'item/003061244',
 'item/00306

## Get HTMLs from paper_urls

In [17]:
# Fetch paper_urls
LIMIT = 10
conn = sqlite3.connect("../saopaulo.db")
with conn:
    result = conn.execute("SELECT * FROM urls LIMIT (?)", (LIMIT,))
    rows = result.fetchall()
conn.close()

In [18]:
rows

[(1, 'paper_urls', 'item/003040726', '20220606', 1, 0),
 (2, 'paper_urls', 'item/002902421', '20220606', 1, 0),
 (3, 'paper_urls', 'item/003036330', '20220606', 1, 0),
 (4, 'paper_urls', 'item/003033960', '20220606', 1, 0),
 (5, 'paper_urls', 'item/003040864', '20220606', 1, 0),
 (6, 'paper_urls', 'item/002861284', '20220606', 1, 0),
 (7, 'paper_urls', 'item/002784581', '20220606', 1, 0),
 (8, 'paper_urls', 'item/002765260', '20220606', 1, 0),
 (9, 'paper_urls', 'item/002781231', '20220606', 1, 0),
 (10, 'paper_urls', 'item/002774116', '20220606', 1, 0)]

In [33]:
# Complete URL
async def fetch_author(session, url):
    async with session.get(url) as response:
        html = await response.text()
        return {'url':url, 'html':html}
    
async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = (fetch_author(session, url) for url in urls)
        results = await asyncio.gather(*tasks)
        return results

In [34]:
# Execute
url_root = 'https://repositorio.usp.br/'
urls = [url_root + row[2] for row in rows]

In [35]:
results = await fetch_all(urls)

In [36]:
results

[{'url': 'https://repositorio.usp.br/item/003040726',
  'html': '\n<!DOCTYPE html>\n<html lang="pt-br" dir="ltr">\n<head>\n    <!-- Altmetric Script -->\n    <script type=\'text/javascript\' src=\'https://d1bxh8uas1mnw7.cloudfront.net/assets/embed.js\'></script>\n\n    <!-- PlumX Script -->\n    <script src="https://cdn.plu.mx/widget-popup.js" integrity="sha256-AXguJKYxgZY9FzwZE8U8EZxUQjcYT6iSQLLGiZTVW84=" crossorigin="anonymous"></script>\n\n  \n    <link rel="shortcut icon" href="//repositorio.usp.br/inc/images/faviconUSP.ico" type="image/x-icon">\n\n    <script src="//repositorio.usp.br/inc/js/jquery.min.js"></script>\n    <script src="//repositorio.usp.br/inc/js/jquery-ui.js"></script>\n    <link rel="stylesheet" href="//repositorio.usp.br/inc/js/jquery-ui.css">\n    <script src="//repositorio.usp.br/inc/js/jquery.form-validator.min.js"></script>\n\n    <!-- Uikit - Local -->\n    <link rel="stylesheet" href="//repositorio.usp.br/inc/uikit/css/uikit.min.css" />\n    <script src="//

In [37]:
# Fetch HTML
print(results[0]['url'])

https://repositorio.usp.br/item/003040726


In [38]:
html_doc = results[0]['html']
soup = BeautifulSoup(html_doc, 'html5lib')

## Parse paper_url HTML and get ORCID

In [39]:
# Extract ORCID and author url
content = soup.select('article article ul li:-soup-contains("Authors: ") ul li')
a_tags = content[0].find_all('a')
url_stem = a_tags[0]['href'][2:]

In [44]:
content

[<li><a class="link" href='//repositorio.usp.br/result.php?filter[]=author.person.name:"Kiyono, César Yukishigue"'>Kiyono, César Yukishigue</a></li>,
 <li><a class="link" href='//repositorio.usp.br/result.php?filter[]=author.person.name:"Perez, N"'>Perez, N</a></li>,
 <li><a class="link" href='//repositorio.usp.br/result.php?filter[]=author.person.name:"Silva, Emílio Carlos Nelli"'>Silva, Emílio Carlos Nelli</a> <a class="link" href="https://orcid.org/0000-0003-1715-1713"><img src="https://orcid.org/sites/default/files/images/orcid_16x16.png"/></a></li>]

In [30]:
# Author url
author_url = url_stem.split('/')[1]
author_url = author_url.replace(" ", "%20")
author_url = author_url.replace("\"", "%22")

In [31]:
author_url

'result.php?filter[]=author.person.name:%22Kiyono,%20César%20Yukishigue%22'

In [32]:
# ORCID
orcid = a_tags[1]['href'].split('/')[-1]

IndexError: list index out of range

In [194]:
author_url

'https://repositorio.usp.br/result.php?filter[]=author.person.name:%22Alves,%20Joner%20Oliveira%22'

## Get ORCIDs from papers

In [14]:
from contextlib import closing

with closing(sqlite3.connect('../saopaulo.db')) as conn:
    with conn:
        result = conn.execute("SELECT * FROM urls LIMIT 10;")
        rows = result.fetchall()
        
rows

[(1, 'paper_urls', 'item/003040726', '20220606', 1, 0),
 (2, 'paper_urls', 'item/002902421', '20220606', 1, 0),
 (3, 'paper_urls', 'item/003036330', '20220606', 1, 0),
 (4, 'paper_urls', 'item/003033960', '20220606', 1, 0),
 (5, 'paper_urls', 'item/003040864', '20220606', 1, 0),
 (6, 'paper_urls', 'item/002861284', '20220606', 1, 0),
 (7, 'paper_urls', 'item/002784581', '20220606', 1, 0),
 (8, 'paper_urls', 'item/002765260', '20220606', 1, 0),
 (9, 'paper_urls', 'item/002781231', '20220606', 1, 0),
 (10, 'paper_urls', 'item/002774116', '20220606', 1, 0)]

In [16]:
url_root = "https://repositorio.usp.br/"
url = url_root + rows[0][2]
url


'https://repositorio.usp.br/item/003040726'

# EXTRA CODE

In [1]:
import requests
from requests_html import HTMLSession

session = HTMLSession()

url = "https://repositorio.usp.br/result.php?filter[]=unidadeUSP:%22EP%22"
r = session.get(url)

In [43]:
r

<Response [200]>

In [44]:
# articles = r.text.select('article article')

In [45]:
# articles

In [6]:
authors = articles[0].find('p')[1]

In [7]:
authors

<Element 'p' class=('uk-article-meta', 'uk-margin-remove') style='color: #666'>

In [8]:
authors.html

'<p class="uk-article-meta uk-margin-remove" style="color: #666"><a class="link" href="result.php?filter[]=author.person.name:&quot;Oliveira, Christian J&#xFA;nior de&quot;">Oliveira, Christian Júnior de</a>; <a class="link" href="result.php?filter[]=author.person.name:&quot;Santos, Mois&#xE9;s Teles dos&quot;">Santos, Moisés Teles dos</a> <a href="https://orcid.org/0000-0002-8082-7619"><img src="inc/images/orcid_16x16.png"/></a>; <a class="link" href="result.php?filter[]=author.person.name:&quot;Vianna Junior, Ardson dos Santos&quot;">Vianna Junior, Ardson dos Santos</a> <a href="https://orcid.org/0000-0003-0192-7839"><img src="inc/images/orcid_16x16.png"/></a></p>'

In [9]:
authors.find('a')

[<Element 'a' class=('link',) href='result.php?filter[]=author.person.name:"Oliveira, Christian Júnior de"'>,
 <Element 'a' class=('link',) href='result.php?filter[]=author.person.name:"Santos, Moisés Teles dos"'>,
 <Element 'a' href='https://orcid.org/0000-0002-8082-7619'>,
 <Element 'a' class=('link',) href='result.php?filter[]=author.person.name:"Vianna Junior, Ardson dos Santos"'>,
 <Element 'a' href='https://orcid.org/0000-0003-0192-7839'>]

In [10]:
authors.find('a')[2].attrs['href']

'https://orcid.org/0000-0002-8082-7619'

In [11]:
for author in authors.find('a'):
    try:
        print(author.href)

SyntaxError: unexpected EOF while parsing (1236353434.py, line 3)